In [ ]:
# Math
import numpy as np

# Visualization
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

from tqdm import tqdm

plt.style.use('qualitative.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors_dict = {0: colors[0], 1: colors[1], 2: colors[2]}

In [ ]:
# Load data
data = load_iris()
X = data['data'][:,[0,2]]
y = data['target']

X -= np.mean(X, axis=0)

classes = np.unique(y)
nr_classes = len(classes)

In [ ]:
def phi(x, y, d=2, total_ys=3):
    v = np.zeros((d+1)*total_ys)
    xhat = np.concatenate([x, np.array([1.])])
    v[y*(d+1):(y+1)*(d+1)] = xhat
    return v

def loss(y1, y2):
    return np.abs(y1-y2)

In [ ]:
# 1) Choose an initial iterate w1
w = np.random.rand(nr_classes*(X.shape[1]+1))

In [ ]:
def visualize_decision_boundary(w,
                                axis,
                                N: int = 300,
                                xmin: float = np.min(X[:,0]),
                                xmax: float = np.max(X[:,0]),
                                ymin: float = np.min(X[:,1]),
                                ymax: float = np.max(X[:,1])):
    X = np.linspace(xmin, xmax, N)
    Y = np.linspace(ymin, ymax, N)
    X, Y = np.meshgrid(X, Y)
    def func(xx):
        yy = []
        for x in tqdm(xx):
            max_score = -np.infty
            predicted_y = None
            for possible_y in classes:
                score = np.dot(w, phi(x, possible_y))
                if score > max_score:
                    max_score = score
                    predicted_y = possible_y
            yy.append(predicted_y)
        
        return np.array(yy)

    Z = func(np.c_[X.ravel(), Y.ravel()])
    Z = Z.reshape(X.shape)
    axis.contour(X, Y, Z, cmap=matplotlib.colors.ListedColormap(['k', 'k', 'k']), alpha=0.5, linewidths=1)


In [ ]:
plt.figure()
visualize_decision_boundary(w, plt.gca())
plt.scatter(X[:,0], X[:,1], c=[colors_dict[_] for _ in y])

plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title(f"Randomly Initialized SVM")
plt.show()

In [ ]:
number_of_iterations = 10000
reg = 0.001
step_size = 0.1

number_of_samples = X.shape[0]

correct = 0
incorrect = 0

# 2) For k = 1, 2, ...
for k in range(number_of_iterations):
    # 3) Select an example (x,y) uniformly at random
    selected_ix = np.random.randint(number_of_samples)
    selected_X = X[selected_ix, :]
    selected_y = y[selected_ix]

    # Naive implementation of Argmax(loss(y, y') + <w, phi(x, y')>)
    max_score = -np.infty
    predicted_y = None
    for possible_y in classes:
        score = loss(selected_y, possible_y) + np.dot(w, phi(selected_X, possible_y))
        if score > max_score:
            max_score = score
            predicted_y = possible_y
    
    # 4) Compute subgradient
    g = reg*w \
        - phi(selected_X, selected_y) \
        + phi(selected_X, predicted_y)

    # 5) Set new iterate
    w -= step_size*g
    
    if predicted_y == selected_y:
        correct += 1
    else:
        incorrect += 1

plt.figure()
visualize_decision_boundary(w, plt.gca())
plt.scatter(X[:,0], X[:,1], c=[colors_dict[_] for _ in y])
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title(f"Trained SVM")
plt.show()
